下载并解压数据，如下所示
cd ~/Downloads
mkdir jena_climate
cd jena_climate
wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip unzip jena_climate_2009_2016.csv.zip

In [ ]:
# 观察数据集的数据
import os
data_dir = '/home/yahu/demos/python/data'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

data = None
with open(fname) as f:
    data = f.read()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
print(header)
print(len(lines))

In [ ]:
# 解析数据, 将数据转换成Numpy数组
import numpy as np

float_data = np.zeros((len(lines), len(header)-1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i,:] = values

In [ ]:
# 绘制温度时间序列
from matplotlib import pyplot as plt

temp = float_data[:,1]
plt.plot(range(len(temp), temp))
# 每10分钟记录一个数据, 每天有144个数据点
# 绘制前10天的温度时间序列
plt.plot(range(1440), temp[:1440])

In [ ]:
# data preprocess
# 数据标准化 减去均值除以标准差
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

# @data: 原始数据
# @lookback: 包括过去多少个时间步
# @delay: 目标应该在未来多少个时间步之后
# @min_index, max_index: 数组中的索引, 界定需要抽取哪些时间步. 有助于划分验证集和测试集
# @step: 数据采样的周期, 默认为6, 即每小时抽取一个数据点
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    # 预测最后一个数据要用到哪个时间步
    if max_index is None:
        max_index = len(data) - delay - 1
    # 第一个目标要用到的最大时间步
    i = min_index + lookback
    while True:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arrange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [ ]:
# 实例化三个生成器, 分别用于训练, 验证和测试
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    shuffle=True,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     shuffle=True,
                     step=step,
                     batch_size=batch_size)

val_steps = (300000 - 200001 - lookback) // batch_size
test_steps = (len(float_data) - 300001 - lookback) // batch_size

利用黑盒深度学习模型解决问题之前, 先尝试一种基于常识的简单方法. 用它来作为合理性检查, 还可以建立一个基准, 更高级的机器学习模型需要打败这个基准才能用.

In [ ]:
# 假设温度时间序列是连续的, 且每天周期性的变化. 因此, 假设24小时后的温度等于现在的温度
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))

evaluate_naive_method()

在开始研究复杂且计算代价很好的模型之前, 先尝试使用简单且计算代价较低的机器学习模型, 比如小型的密集连接网络. 保证进一步增加问题的复杂度是合理的.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import RMSprop

model = Sequential()
model.add(Flatten(input_shape=(lookback // step, float_data.shape[-1])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')

history = model.fit_generator(train_gen,
                    steps_per_epoch=500,
                    epochs=20,
                    validation_data=val_gen,
                    validation_steps=val_steps)

In [ ]:
import utils
utils.draw_acc_and_loss(history)

简单的全连接方法从输入数据中删除了时间的概念. 数据是一个序列, 其中的因果关系和顺序都很重要. 下面使用一种循环序列处理模型.

In [ ]:
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import RMSprop

model = Sequential()
model.add(GRU(32, input_shape=(None, float_data.shape[-1])))
model.add(Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
model.fit_generator(train_gen,
                    steps_per_epoch=500,
                    epochs=20,
                    validation_data=val_gen,
                    validation_steps=val_steps)

In [ ]:
import utils
utils.draw_acc_and_loss(history)

使用循环dropout降低过拟合, 可以打破该层训练数据中的偶然相关性.
在循环层前面使用dropout会妨碍学习过程. 循环神经网络中, 每个时间步应该使用相同的dropout掩码. 这可以让网络沿着时间步正确地传播其学习误差, 随时间随机变化的dropout掩码会破坏这个误差信号,不利于学习过程.

In [ ]:
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import RMSprop

model = Sequential()
model.add(GRU(32,
              dropout=0.2,              # rate of inputs dropout
              recurrent_dropout=0.2,    # rate of recurrent dropout
              input_shape=(None, float_data.shape[-1])))
model.add(Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
model.fit_generator(train_gen,
                    steps_per_epoch=500,
                    epochs=20,
                    validation_data=val_gen,
                    validation_steps=val_steps)
utils.draw_acc_and_loss(history)

In [ ]:
# stack recurrent layers
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import RMSprop

model = Sequential()
model.add(GRU(32,
              dropout=0.2,              # rate of inputs dropout
              recurrent_dropout=0.2,    # rate of recurrent dropout
              input_shape=(None, float_data.shape[-1])))
model.add(GRU(64, activation='relu',
              dropout=0.1,
              recurrent_dropout=0.5))
model.add(Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                    steps_per_epoch=500,
                    epochs=20,
                    validation_data=val_gen,
                    validation_steps=val_steps)
utils.draw_acc_and_loss(history)

In [ ]:
# 将双向GRU用于温度预测任务
model = Sequential()
model.add(Bidirectional(GRU(32), input_shape=(None, float_data.shape[-1])))
model.add(Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)
utils.draw_acc_and_loss(history)

更多的提高性能的方法:
- 调节堆叠层中每层的单元个数
- 调节优化器的学习率
- 使用LSTM替代GRU
- 在循环层上面使用更大的密集连接回归器, 即更大的Dense层或Dense层的堆叠
- 在测试集上运行性能最佳的模型, 否则模型将对验证集过拟合

总结:
- 建立一个基于常识的基准
- 从简单的模型开始, 以证明增加计算代价是有意义的
- 如果时间顺序对数据很重要, 循环网络是种很合适的方法, 一般会好过将时间数据展平的模型
- 循环网络中的dropout应该使用不随时间变化的dropout掩码和循环dropout掩码
- 堆叠RNN的表示能力更强, 但计算代价更高,在较简单的问题上可能不一定有用
- 双向RNN从两个方向查看一个序列, 对NLP问题非常有用. 但如果序列数据中, 最近的数据比开头的数据包含更多的信息, 那么这种方法效果就不明显.